In [1]:
import numpy as np
import face_detection
import os, python_speech_features
import scipy.io.wavfile as wav
import pandas as pd
import torch.nn.functional as F
from talkNet import talkNet
from dataset import MyDataset
from torch.utils.data import DataLoader
import tqdm
import torch
import cv2
import tools
import subprocess
from collections import defaultdict
import whisper
detector = face_detection.build_detector(
"DSFDDetector", confidence_threshold=.3, nms_iou_threshold=.5) #DSFDDetector

c:\Users\jmmol\miniconda3\envs\alc\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\jmmol\miniconda3\envs\alc\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [2]:
#inputVideo = "C:/Users/jmmol/Desktop/LIP-RTVE/MP4s/speaker092/speaker092_0002.mp4"
inputVideo = r"inputs/dos_personas_hablando_por_turnos.mp4"
videoDuration = tools.checkVideoDuration(inputVideo)
videoFrames = int(videoDuration*25)
#tools.splitVideo(inputVideo,2,videoDuration)

In [3]:
#Carga vídeo
res, facePos, faceFrames = tools.saveMultiFace(inputVideo,detector,50)
# AUDIO PROCESSING
audioPath = tools.convert_video_to_audio_ffmpeg(inputVideo)
_,sig = wav.read(audioPath)#r"C:\Users\jmmol\Desktop\COSAS V7\TFM\speaker042_0063.wav") #
# _,sig = wav.read(r"C:\Users\jmmol\Desktop\COSAS V7\TFM\speaker000_0000.wav") 
# audioPath = r"C:\Users\jmmol\Desktop\COSAS V7\TFM\speaker000_0000.wav"
audio = python_speech_features.mfcc(sig, 16000, numcep = 13, winlen = 0.025, winstep = 0.010) #ASUME VIDEO A 25 Y AUDIO A 100, MODIFICAR
print("Nº CARAS:",len(res))
for cara in res.keys():
    print(len(res[cara]))


Nº CARAS: 3
244
244
3


In [4]:
print(faceFrames[2])
# cv2.imshow("My Video", res[3][50])
# cv2.waitKey(0)

[24, 45, 46]


In [11]:
model = talkNet()
#model.load_state_dict(torch.load("./exps/exp2/model/model_0002.model"))
model.load_state_dict(torch.load("./exps/exp1/model/model21_0006.model"))
windowSize = 21

06-16 10:30:17 Model para number = 15.01


In [12]:
videoDir = "C:/Users/jmmol/Desktop/COSAS V7/TFM/npz"
audioDir = "C:/Users/jmmol/Desktop/COSAS V7/TFM/mfccs"
datasetTrain = MyDataset(51,videoDir,audioDir,"testSamples.csv")
item = datasetTrain.__getitem__(4309)
valLoader = DataLoader(dataset=datasetTrain,shuffle=False,batch_size=32,num_workers=14) #Cambiar num_workers

25 100


In [13]:
# correctFrames, totalFrames = 0, 0
# correctSamples, totalSamples = 0, 0
# totalPreds = []
# for num, (audioFeature, visualFeature, labels) in enumerate(tqdm.tqdm(valLoader)):
#     with torch.no_grad():    
#         output = model((audioFeature,visualFeature))
#         labels = labels.cuda()
#         batchPreds = torch.reshape(output[1], labels.shape)
#         #Precision a nivel de video
#         videoPreds = torch.mode(batchPreds,dim=1)[0]
#         totalPreds.extend(videoPreds.detach().cpu().numpy().astype(int))
#         labelMode = torch.mode(labels,dim=1)[0]
#         correctSamples += (videoPreds == labelMode).sum().float()
#         totalSamples += len(labels)
#         # Precision a nivel de frame
#         labels = labels.reshape((-1))
#         correctFrames += (output[1] == labels).sum().float()
#         totalFrames += len(labels) 
# #7865
# print("Frame acc:", correctFrames/totalFrames)
# print("Sample acc:", correctSamples/totalSamples)
center = 47
iAudio = tools.padAudio(audio,1,center,51,len(res[0]))
iVideo = tools.padVideo(res[0],center,51)
print(iAudio[40],item[0][40])
print("Audios iguales?",(iAudio == item[0]).sum(),iAudio.shape[0]*iAudio.shape[1])
print("Videos iguales?",(iVideo == item[1]).sum(),iVideo.shape[0]*112*112)
print(audio[0],item[0][0])

tensor([ 16.1682,  -1.0144,   0.7126, -13.8878,  -1.9008, -15.4209,  -7.3984,
         10.0662, -12.3333,  -1.0726,  -5.4619, -14.3538,  -9.1944]) tensor([ 16.0407,   5.1092,  -7.4174,  26.3697, -21.4168, -34.9214, -26.8578,
        -27.1986, -11.6547,  -2.2427, -25.6342, -12.2112,   4.5576])
Audios iguales? tensor(0) 2652
Videos iguales? tensor(3866) 639744
[ 18.97219561  -0.06125094 -18.13477886 -21.8923113  -25.36991588
   0.57649002  -3.81110727  -7.30372566  12.85849014  21.97313694
   9.50819258 -22.2504338  -30.71788893] tensor([ 11.6267,   6.2778,   2.7260,  -5.3083,  -9.2899,  -7.2238,  -8.3037,
         -3.4917,  -6.4616, -10.6301, -17.5955,  -8.8042,  -3.2956])


In [14]:
predArray = defaultdict(list)
# MEDIA FRAMES
# for actualSpeaker in res.keys():
#     print("SPEAKER", actualSpeaker)
#     for i in range(len(res[actualSpeaker])):
#         #center = i
#         center = faceFrames[actualSpeaker][i]
#         print("FRAME Nº ",center)
#         iAudio = tools.padAudio(audio,1,center,windowSize,videoFrames).unsqueeze(0)
#         iVideo = tools.padVideo(res[actualSpeaker],center-faceFrames[actualSpeaker][0],windowSize).unsqueeze(0)
#         #print("audio shape: ", audio.shape)
#         #print("video shape: ", video.shape)
#         # iAudio = item[0].unsqueeze(0)
#         # iVideo = item[1].unsqueeze(0)
#         print(iAudio[0][0][0:5],iVideo[0][0][0][0:5])
#         output = model((iAudio,iVideo))
#         #output = model((audio,video))
#         #print(output)
#         print("Pred ->",torch.mode(output[1])[0],"\tProb Pos ->",torch.mean(output[1]))
#         predArray[actualSpeaker].append(torch.mean(output[1]).detach().cpu().numpy())

# SECUENCIAL
sideWindowSize = int((windowSize-1)/2)
print("sidewindow:",sideWindowSize)
for actualSpeaker in res.keys():
    print("SPEAKER", actualSpeaker)
    for i in range(0,len(res[actualSpeaker])+windowSize,windowSize):
        #center = i
        center = faceFrames[actualSpeaker][0]+i
        #print("FRAME Nº ",center)
        iAudio = tools.padAudio(audio,1,center,windowSize,videoFrames).unsqueeze(0)
        iVideo = tools.padVideo(res[actualSpeaker],center-faceFrames[actualSpeaker][0],windowSize).unsqueeze(0)
        #print(iAudio[0][0][0:5],iVideo[0][0][0][0:5])
        output = model((iAudio,iVideo))
        predArray[actualSpeaker].extend(output[1].detach().cpu().numpy().tolist())
    # Delete < 0 frame predictions (initial)
    predArray[actualSpeaker] = predArray[actualSpeaker][sideWindowSize:len(res[actualSpeaker])+sideWindowSize]
for actualSpeaker in res.keys():
    print(len(predArray[actualSpeaker]))


sidewindow: 10
SPEAKER 0
SPEAKER 1
SPEAKER 2
244
244
3


In [15]:
def createVideo(videoName,imgFrames,audioPath,width,height):
    video = cv2.VideoWriter("output.mp4", 0, 25, (width,height))

    for image in imgFrames:
        video.write(image)

    cv2.destroyAllWindows()
    video.release()
    print(' '.join(["ffmpeg","-y","-i",os.getcwd()+"/output.mp4","-i",audioPath,"-map","0:v","-map","1:a", "-c:v", "copy", "-shortest", os.getcwd()+"/output2.mp4"]))
    res = subprocess.check_output(["ffmpeg","-y","-i",os.getcwd()+f"/output.mp4","-i",audioPath,"-map","0:v","-map","1:a", "-c:v", "copy", "-shortest", os.getcwd()+f"/outputs/videos/{videoName}"])
    os.remove(os.getcwd()+f"/output.mp4")
    
    print(res)

In [16]:
#Asignación de bounding boxes y probabilidades al video de output
os.makedirs("outputs/videos", exist_ok=True)
cap = cv2.VideoCapture(inputVideo)
videoImages = []
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

while True:
    ret, image = cap.read()
    videoImages.append(image)
    if ret == False:
        break


for speakerN in predArray.keys():
    for i, fr in enumerate(faceFrames[speakerN]): #Para cada frame en el que aparezca la cara
        try:
            image = videoImages[fr]
            greenValue = int(255*predArray[speakerN][i])
            redValue = 255-greenValue
            color = (0,greenValue,redValue)
            #print(color)
            print(speakerN,i)
            xmin,ymin,xmax,ymax = facePos[speakerN][i]
            image = cv2.rectangle(image, (xmin,ymin), (xmax,ymax),color , 1)
            cv2.putText(image, str(predArray[speakerN][i]), (xmin, ymin-5), cv2.FONT_HERSHEY_SIMPLEX, 0.35, color, 1)
        except:
            pass

        # cv2.imshow("My Video", image)
        # cv2.waitKey(0)


cap.release()
cv2.destroyAllWindows()
print(len(videoImages))
path = os.path.normpath(inputVideo)
videoName = str(path.split(os.sep)[-1])
createVideo(videoName,videoImages,audioPath,width,height)
#print(predArray[3])

0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
0 11
0 12
0 13
0 14
0 15
0 16
0 17
0 18
0 19
0 20
0 21
0 22
0 23
0 24
0 25
0 26
0 27
0 28
0 29
0 30
0 31
0 32
0 33
0 34
0 35
0 36
0 37
0 38
0 39
0 40
0 41
0 42
0 43
0 44
0 45
0 46
0 47
0 48
0 49
0 50
0 51
0 52
0 53
0 54
0 55
0 56
0 57
0 58
0 59
0 60
0 61
0 62
0 63
0 64
0 65
0 66
0 67
0 68
0 69
0 70
0 71
0 72
0 73
0 74
0 75
0 76
0 77
0 78
0 79
0 80
0 81
0 82
0 83
0 84
0 85
0 86
0 87
0 88
0 89
0 90
0 91
0 92
0 93
0 94
0 95
0 96
0 97
0 98
0 99
0 100
0 101
0 102
0 103
0 104
0 105
0 106
0 107
0 108
0 109
0 110
0 111
0 112
0 113
0 114
0 115
0 116
0 117
0 118
0 119
0 120
0 121
0 122
0 123
0 124
0 125
0 126
0 127
0 128
0 129
0 130
0 131
0 132
0 133
0 134
0 135
0 136
0 137
0 138
0 139
0 140
0 141
0 142
0 143
0 144
0 145
0 146
0 147
0 148
0 149
0 150
0 151
0 152
0 153
0 154
0 155
0 156
0 157
0 158
0 159
0 160
0 161
0 162
0 163
0 164
0 165
0 166
0 167
0 168
0 169
0 170
0 171
0 172
0 173
0 174
0 175
0 176
0 177
0 178
0 179
0 180
0 181
0 182
0 183
0 184


In [21]:
# GUARDAR DATOS 
# COGER SOLO FRAGMENTO DE AUDIO EN EL QUE SE DETECTA HABLANTE
model = whisper.load_model("small")
transcription = model.transcribe(audioPath, language="es", verbose=True)
os.makedirs("outputs/npz", exist_ok=True)
np.savez_compressed(r"outputs/npz/"+videoName,facePos=facePos,faceFrames=faceFrames,preds=predArray,transcription=transcription)


[00:00.000 --> 00:05.280]  Hace 25 años la ONU declaró el 3 de diciembre como Día Internacional de las Personas con
[00:05.280 --> 00:09.440]  Discapacidad, con el objetivo de concienciarnos a todos sobre su situación y, en su caso,
[00:09.440 --> 00:09.440]  el 3 de diciembre.


In [12]:
#createVideo(res[3],audioPath,112,112)

[00:00.000 --> 00:05.280]  Hace 25 años la ONU declaró el 3 de diciembre como Día Internacional de las Personas con
[00:05.280 --> 00:09.440]  Discapacidad, con el objetivo de concienciarnos a todos sobre su situación y, en su caso,
[00:09.440 --> 00:09.440]  el 3 de diciembre.


In [26]:
loaded = np.load(r"C:\Users\jmmol\Desktop\COSAS V7\TFM\outputs\npz\speaker270_0002.mp4.npz",allow_pickle=True)

In [27]:
loaded["transcription"]

array({'text': ' Hace 25 años la ONU declaró el 3 de diciembre como Día Internacional de las Personas con Discapacidad, con el objetivo de concienciarnos a todos sobre su situación y, en su caso,', 'segments': [{'id': 0, 'seek': 0, 'start': 0.0, 'end': 5.28, 'text': ' Hace 25 años la ONU declaró el 3 de diciembre como Día Internacional de las Personas con', 'tokens': [50364, 389, 617, 3552, 11424, 635, 9299, 52, 16694, 812, 806, 805, 368, 14285, 49772, 2617, 413, 2686, 4844, 13608, 368, 2439, 8443, 296, 416, 50628], 'temperature': 0.0, 'avg_logprob': -0.5163383796566823, 'compression_ratio': 1.3311258278145695, 'no_speech_prob': 0.09169353544712067}, {'id': 1, 'seek': 0, 'start': 5.28, 'end': 9.44, 'text': ' Discapacidad, con el objetivo de concienciarnos a todos sobre su situación y, en su caso,', 'tokens': [50628, 4208, 9485, 326, 4580, 11, 416, 806, 29809, 368, 416, 537, 30322, 24979, 257, 6321, 5473, 459, 29343, 288, 11, 465, 459, 9666, 11, 50836], 'temperature': 0.0, 'avg_logprob'